In [1]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import pi,h
from qutip import *
import functools
import operator

Using matplotlib backend: TkAgg


# Two-level qubit with SFQ pulse train

## Using a SFQ pulse train in H1(t)

In [2]:
# check a Gaussian pulse train

# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 90

t1 = np.linspace(0, n * 2*pi/w_01, 100001)
y1 = functools.reduce(operator.add, [ np.exp(-(t1-tc -i*2*pi/w_01)**2/2/tau**2) * (np.heaviside(t1-i*2*pi/w_01,0)- np.heaviside(t1-2*tc-i*2*pi/w_01,0)) for i in range(n)])

plt.plot(t1, y1)

In [3]:
def H1_coeff(t, args):
    out = functools.reduce(operator.add, 
        [np.exp(-(t-args['tc']-i*2*pi/args['w_01'])**2/2/args['tau']**2) 
         * (np.heaviside(t-i*2*pi/args['w_01'],0) + np.heaviside(t+2*args['tc']-i*2*pi/args['w_01'],0)) 
         for i in range(args['n'])])
    
    return out

In [4]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 90

# solve Lindblad master equation
H0 = w_01/2*(qeye(2) - sigmaz()) 
H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay()
H = [H0, [H1, H1_coeff]]

psi0 = basis(2,0)
tlist_pulseTrain = np.linspace(0, n * 2*pi/w_01, 501)
args = {'tau':tau, 'tc': tc, 'w_01': w_01,'n':n}

output = mesolve(H, psi0, tlist_pulseTrain, [], [sigmaz()], args, progress_bar=True)  

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist_pulseTrain, output.expect[0])
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

10.2%. Run time:   1.77s. Est. time left: 00:00:00:15
20.2%. Run time:   3.65s. Est. time left: 00:00:00:14
30.1%. Run time:   5.20s. Est. time left: 00:00:00:12
40.1%. Run time:   6.64s. Est. time left: 00:00:00:09
50.1%. Run time:   8.21s. Est. time left: 00:00:00:08
60.1%. Run time:   9.74s. Est. time left: 00:00:00:06
70.1%. Run time:  11.21s. Est. time left: 00:00:00:04
80.0%. Run time:  12.68s. Est. time left: 00:00:00:03
90.0%. Run time:  14.38s. Est. time left: 00:00:00:01
Total run time:  15.88s


## Using evolution operator : faster

In [133]:
def H1_coeff_1(t, args):
    """ Time-dependant coefficient of H1 Hamiltonian """
    return np.exp(-t**2/2/args['tau']**2)

def U_f(t):
    """ free time evolution operator """
    return (1j*w_01*t/2*sigmaz()).expm() 

def U_single_multiplier(U_single, n):
    U = U_single
    counter = 1
    while True:
        if counter > n: return
        yield U 
        U *= U_single
        counter += 1

In [143]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 4e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 180

# Compute evolution operator over one unit of SFQ pulse train,
# i.e., one SFQ + free evolotuion
H0 = w_01/2*(qeye(2) - sigmaz()) 
H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay() 
H = [H0, [H1, H1_coeff_1]]

tlist = np.linspace(-tc, tc, 101)
args = {'tau':tau, 'tc':tc, 'w_01':w_01}

U_single_Gauss = propagator(H, tlist, [], args=args) # time evolution during a Gaussian pulse

U_single = U_f(2*pi/w_01-tc) * U_single_Gauss[-1] * U_f(-tc) # Eq.26 in PRApplied paper.

U_G_list = [i for i in U_single_multiplier(U_single, n)]

# U_G_list = [U_single]
# for i in range(n-1):
#     U_G_list.append(U_single*U_G_list[-1])

sigmaz_expect = expect(sigmaz(), U_G_list*basis(2,0)) # expectation value of sigma_z
sigmay_expect = expect(sigmay(), U_G_list*basis(2,0))
sigmax_expect = expect(sigmax(), U_G_list*basis(2,0))

# plot
tlist = [2*pi/w_01*i for i in range(n)]
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, sigmaz_expect, label='<z>')
ax.plot(tlist, sigmay_expect, label='<y>')
ax.plot(tlist, sigmax_expect, label='<x>')
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

In [140]:
U_G[-1]

array([[-0.56850789-0.4130849j , -0.575601  -0.41814179j]])

## Fidelity vs Number of SFQ pulses 
As in Fig.4

In [137]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 4e-3 # in ns 
tc = 5*tau # in ns 
theta = pi/2

H0 = w_01/2*(qeye(2) - sigmaz()) 
U_id = (-1j*theta*sigmay()/2).expm()

N_list = np.arange(1, 300, 1)
fid_list = []
UG_list = []
for n in N_list:
    del_theta = theta / float(n)
    H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay() 
    H = [H0, [H1, H1_coeff_1]]
    
    tlist = np.linspace(-tc, tc, 101)
    args = {'tau':tau, 'tc':tc, 'w_01':w_01}

    U_single_Gauss = propagator(H, tlist, [], args=args) # time evolution during a Gaussian pulse

    U_single = U_f(2*pi/w_01-tc) * U_single_Gauss[-1] * U_f(-tc) # Eq.26 in PRApplied paper.

    U_G = U_single**n
    UG_list.append(U_G) # optional. for debugging
    
    Fid_n = (np.absolute((U_id.dag()*U_G).tr())**2 + 2) / 6
    fid_list = np.append(fid_list,Fid_n)


# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(N_list, 1 - fid_list)
ax.set_xlabel('Number of SFQ pulses', fontsize=18)
ax.set_ylabel('Gate error, 1-F', fontsize=18)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')